In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster("local").setAppName("WordCount")
sc = SparkContext(conf = conf)

In [ ]:
# map과 flatmap의 차이를 확실히 알아둘 것~

In [2]:
input = sc.textFile("../data/book.txt")

In [3]:
words = input.flatMap(lambda x: x.split())

In [5]:
words.collect()[:5]

['Self-Employment:', 'Building', 'an', 'Internet', 'Business']

In [6]:
wordCounts = words.countByValue()

In [14]:
#wordCounts # defaultDict으로 나오게 됩니다!

In [15]:
for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

## 결과를 보면 단어의 우측에 ", ' 이런 불순물들이 껴있음..!

# 정규 표현식을 사용해보자

In [16]:
import re

In [17]:
def normalizeWords(text):
    return re.compile(r'\W+', re.UNICODE).split(text.lower())

In [18]:
input = sc.textFile("../data/book.txt")
words = input.flatMap(normalizeWords)
# wordCounts = words.countByValue()

In [ ]:
for word, count in wordCounts.items():
    cleanWord = word.encode('ascii', 'ignore')
    if (cleanWord):
        print(cleanWord.decode() + " " + str(count))

In [ ]:
input = sc.textFile("../data/book.txt")
words = input.flatMap(normalizeWords)
# wordCounts = words.countByValue()

In [22]:
wordCounts = words.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey()
results = wordCountsSorted.collect()

In [24]:
for result in results:
    count = str(result[0])
    word = result[1].encode('ascii', 'ignore')
    if (word):
        print(word.decode() + ":\t\t" + count)